In [1]:
import torch
import torchvision.models as models

# Load a pre-trained VGG16 model
vgg_model = models.vgg16(pretrained=True)

c:\Users\iboba\Desktop\final_project\.conda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\iboba\Desktop\final_project\.conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
num_features = vgg_model.classifier[6].in_features
features = list(vgg_model.classifier.children())[:-1] # Remove last layer
features.extend([torch.nn.Linear(num_features, 2)]) # Add our layer with 2 outputs
vgg_model.classifier = torch.nn.Sequential(*features)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg_model = vgg_model.to(device)

In [4]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'gender_classification/Training': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'gender_classification/Validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [11]:
image_datasets = {x: ImageFolder(f"./{x}", data_transforms[x])
                  for x in ['gender_classification/Training', 'gender_classification/Validation']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4,
                             shuffle=True, num_workers=4)
              for x in ['gender_classification/Training', 'gender_classification/Validation']}

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg_model.parameters(), lr=0.001, momentum=0.9)

In [7]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['gender_classification/Training', 'gender_classification/Validation']:
            if phase == 'gender_classification/Training':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # Move the inputs and labels to the same device as the model
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'gender_classification/Training'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize in training phase
                    if phase == 'gender_classification/Training':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

In [8]:
def check_accuracy(model, phase):
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloaders[phase]:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / len(image_datasets[phase])
    print(f'{phase} Acc: {acc:.4f}')

In [9]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
# This 

In [13]:
# Train and evaluate
vgg_model = train_model(vgg_model, criterion, optimizer, num_epochs=10)
# This model is based on VGG16 but with a different classifier layer (2 outputs instead of 1000)

Epoch 0/9
----------
gender_classification/Training Loss: 0.7767 Acc: 0.5714
gender_classification/Validation Loss: 0.7299 Acc: 0.4571
Epoch 1/9
----------
gender_classification/Training Loss: 0.7946 Acc: 0.5102
gender_classification/Validation Loss: 0.7351 Acc: 0.5571
Epoch 2/9
----------
gender_classification/Training Loss: 0.7946 Acc: 0.3673
gender_classification/Validation Loss: 0.7002 Acc: 0.5571
Epoch 3/9
----------
gender_classification/Training Loss: 0.6618 Acc: 0.6531
gender_classification/Validation Loss: 0.7207 Acc: 0.4571
Epoch 4/9
----------
gender_classification/Training Loss: 0.6590 Acc: 0.6735
gender_classification/Validation Loss: 0.6683 Acc: 0.5714
Epoch 5/9
----------
gender_classification/Training Loss: 0.7287 Acc: 0.4898
gender_classification/Validation Loss: 0.7318 Acc: 0.5571
Epoch 6/9
----------
gender_classification/Training Loss: 0.7622 Acc: 0.5714
gender_classification/Validation Loss: 0.6628 Acc: 0.6857
Epoch 7/9
----------
gender_classification/Training Los